# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ribeira Grande,PT,38.5167,-28.7000,64.80,94,75,13.00,light intensity drizzle
1,1,Jamestown,US,42.0970,-79.2353,28.02,49,75,13.80,broken clouds
2,2,Castro,BR,-24.7911,-50.0119,69.84,63,15,4.70,few clouds
3,3,Mahebourg,MU,-20.4081,57.7000,70.09,83,20,3.44,few clouds
4,4,Provideniya,RU,64.3833,-173.3000,25.56,84,100,6.11,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip?"))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?65
What is the maximum temperature you would like for your trip?85


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(15)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Castro,BR,-24.7911,-50.0119,69.84,63,15,4.70,few clouds
3,3,Mahebourg,MU,-20.4081,57.7000,70.09,83,20,3.44,few clouds
5,5,Barra Do Corda,BR,-5.5000,-45.2500,73.26,96,100,4.32,moderate rain
8,8,Rikitea,PF,-23.1203,-134.9692,71.08,67,93,9.28,overcast clouds
9,9,Saint-Philippe,RE,-21.3585,55.7679,71.19,79,24,8.72,few clouds
10,10,Atuona,PF,-9.8000,-139.0333,77.09,74,10,10.89,clear sky
13,13,Kalmunai,LK,7.4167,81.8167,75.27,88,100,11.16,overcast clouds
16,16,Nizwa,OM,22.9333,57.5333,71.26,46,0,3.80,clear sky
18,18,Vaini,TO,-21.2000,-175.2000,77.07,71,94,6.40,overcast clouds
24,24,Manama,BH,26.2154,50.5832,76.98,57,0,19.57,clear sky


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
preferred_cities_df.count()

City_ID                281
City                   281
Country                281
Lat                    281
Lng                    281
Max Temp               281
Humidity               281
Cloudiness             281
Wind Speed             281
Current Description    281
dtype: int64

In [6]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()
# Display sample data
clean_travel_cities.count()

City_ID                281
City                   281
Country                281
Lat                    281
Lng                    281
Max Temp               281
Humidity               281
Cloudiness             281
Wind Speed             281
Current Description    281
dtype: int64

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [7]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng
2,Castro,BR,69.84,few clouds,-24.7911,-50.0119
3,Mahebourg,MU,70.09,few clouds,-20.4081,57.7000
5,Barra Do Corda,BR,73.26,moderate rain,-5.5000,-45.2500
8,Rikitea,PF,71.08,overcast clouds,-23.1203,-134.9692
9,Saint-Philippe,RE,71.19,few clouds,-21.3585,55.7679


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [8]:
# Add a new column to the hotel_df DataFrame named "Hotel Name"
hotel_df["Hotel Name"] = ""

# Sample the new hotel_df DataFrame
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Castro,BR,69.84,few clouds,-24.7911,-50.0119,
3,Mahebourg,MU,70.09,few clouds,-20.4081,57.7000,
5,Barra Do Corda,BR,73.26,moderate rain,-5.5000,-45.2500,
8,Rikitea,PF,71.08,overcast clouds,-23.1203,-134.9692,
9,Saint-Philippe,RE,71.19,few clouds,-21.3585,55.7679,
10,Atuona,PF,77.09,clear sky,-9.8000,-139.0333,
13,Kalmunai,LK,75.27,overcast clouds,7.4167,81.8167,
16,Nizwa,OM,71.26,clear sky,22.9333,57.5333,
18,Vaini,TO,77.07,overcast clouds,-21.2000,-175.2000,
24,Manama,BH,76.98,clear sky,26.2154,50.5832,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [9]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [10]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data scanned and checked      ")
print("-----------------------------")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data scanned and checked      
-----------------------------


In [11]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Castro,BR,69.84,few clouds,-24.7911,-50.0119,CHACARA BAILLY
3,Mahebourg,MU,70.09,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
5,Barra Do Corda,BR,73.26,moderate rain,-5.5000,-45.2500,Belchior Palace Hotel
8,Rikitea,PF,71.08,overcast clouds,-23.1203,-134.9692,People ThankYou
9,Saint-Philippe,RE,71.19,few clouds,-21.3585,55.7679,Le four à pain


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [12]:
hotel_df['Hotel Name'].replace('',np.nan, inplace=True)

# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
hotel_df.tail(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
678,Pontianak,ID,75.18,few clouds,-0.0333,109.3333,Aston Pontianak Hotel & Convention Center
682,Guinguineo,SN,76.82,clear sky,14.2667,-15.9500,NaN
686,Kualakapuas,ID,73.09,overcast clouds,-3.0091,114.3876,Guest House Izonk
693,Kiama,AU,75.25,clear sky,-34.6833,150.8667,The Sebel Kiama Harbourside
696,Beira,MZ,77.09,clear sky,-19.8436,34.8389,Beira Terrace Hotel
697,Pangai,TO,77.77,clear sky,-19.8000,-174.3500,Ha'apai Beach Resort
700,Samarai,PG,84.40,overcast clouds,-10.6167,150.6667,Nuli Sapi
701,Hualmay,PE,67.55,broken clouds,-11.0964,-77.6139,Hotel Villa Kitzia Huacho
702,Xai-Xai,MZ,75.38,broken clouds,-25.0519,33.6442,Complexo da Familia
703,Tanga,TZ,75.52,light rain,-5.0667,39.1000,Lulu Image Hotel


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [13]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [14]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [15]:
# Review the formatting template provided
info_box_template = """

Hotel Name
{Hotel Name}

City
{City}
Country
{Country}
Current Weather
{Current Description} and {Max Temp} °F


"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [16]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n\nHotel Name\nCHACARA BAILLY\n\nCity\nCastro\nCountry\nBR\nCurrent Weather\nfew clouds and 69.84 °F\n\n\n',
 '\n\nHotel Name\nShandrani Beachcomber Resort & Spa\n\nCity\nMahebourg\nCountry\nMU\nCurrent Weather\nfew clouds and 70.09 °F\n\n\n',
 '\n\nHotel Name\nBelchior Palace Hotel\n\nCity\nBarra Do Corda\nCountry\nBR\nCurrent Weather\nmoderate rain and 73.26 °F\n\n\n',
 '\n\nHotel Name\nPeople ThankYou\n\nCity\nRikitea\nCountry\nPF\nCurrent Weather\novercast clouds and 71.08 °F\n\n\n',
 '\n\nHotel Name\nLe four à pain\n\nCity\nSaint-Philippe\nCountry\nRE\nCurrent Weather\nfew clouds and 71.19 °F\n\n\n',
 '\n\nHotel Name\nVilla Enata\n\nCity\nAtuona\nCountry\nPF\nCurrent Weather\nclear sky and 77.09 °F\n\n\n',
 '\n\nHotel Name\nVS Villa\n\nCity\nKalmunai\nCountry\nLK\nCurrent Weather\novercast clouds and 75.27 °F\n\n\n',
 '\n\nHotel Name\nOrient Apartments\n\nCity\nNizwa\nCountry\nOM\nCurrent Weather\nclear sky and 71.26 °F\n\n\n',
 '\n\nHotel Name\nKeleti Beach Resort\n\nCity\nVai

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [17]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
2,-24.7911,-50.0119
3,-20.4081,57.7000
5,-5.5000,-45.2500
8,-23.1203,-134.9692
9,-21.3585,55.7679
10,-9.8000,-139.0333
13,7.4167,81.8167
16,22.9333,57.5333
18,-21.2000,-175.2000
24,26.2154,50.5832


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [19]:
# Add a marker layer for each city to the map. 
fig=gmaps.figure() 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig.add_layer(marker_layer)

#save file
saveas(fig,WeatherPy_vacation_map.png)
# Display the figure containing the map
fig

NameError: name 'saveas' is not defined